In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Analysis of wrong geometry --> graph conversions 

This notebook has the goal to explore whether the conversion from cartesian geometry (contained in the CONTCAR files of VASP) to graph has been carried out correctly. The algorithm that creates the graph representation is based on Voronoi tesselation and implemented in pyRDTP.
NB The outcome of the conversion from geometry to graph strongly depends on the Voronoi tolerance parameter (default=0.25) and the applied atomic radii (default=CORDERO set of atomic radii).

In [ ]:
def geometry_to_graph_analysis(dataset: str):
    """
    Check that all adsorption samples in the defined dataset are correctly 
    converted to a graph. The correctness of the graph is defined by the presence 
    of at least one metal atom in the system.
    Args: 
        dataset(str): name of the dataset analyzed. Must be located in "./Data/"" directory.
                      The parsed file is Dataset.dat, which contains the graph representations
                      of each sample in COO format.
    Returns:
        new_list(list): list of lists contains the metal atoms in the graph.  
        wrong_graphs(int): number of badly represented samples, in which no metal atom is present in the 
                           graph representation.
        wrong_samples(list): list of the badly represented data
    """
    atoms = ('C', 'H', 'O', 'N', 'S')
    if dataset[:3] == "gas":
        return "Error: This function is just for datasets containing adsorption systems!"
    with open("../data/FG_dataset/{}/pre_05_False_15.dat".format(dataset)) as f:
        all_lines = f.readlines()
    dataset_size = int(len(all_lines)/5)
    lines = []
    labels = []
    for i in range(dataset_size):
        lines.append(all_lines[1 + 5*i])  # Read the second line of each graph (ex. "C H C H Ag")
        labels.append(all_lines[5*i])     # Read label of each sample (ex. "ag-4a01-a")
    for i in range(dataset_size):
        lines[i] = lines[i].strip("\n")
        lines[i] = lines[i].split()
        labels[i] = labels[i].strip("\n")
    new_list = [[]] * dataset_size
    wrong_samples = []
    for i in range(dataset_size):
        new_list[i] = [lines[i][j] for j in range(len(lines[i])) if lines[i][j] not in atoms]
        if new_list[i] == []:
            wrong_samples.append(labels[i])
    wrong_graphs = new_list.count([])
    print("Dataset Name: {}".format(dataset))
    print("Dataset Size: {}".format(dataset_size))
    print("Number of badly represented data: {}".format(wrong_graphs))
    print("Percentage of bad representations: {:.2f}%".format((wrong_graphs/dataset_size)*100))
    return new_list, wrong_graphs, wrong_samples, dataset_size, labels

## Analysis of the global dataset for the GNN

In [ ]:
all_datasets = ["amides", "amidines", "aromatics", "aromatics2",
                 "carbamate_esters", "group2", "group2b", "group3N",
                 "group3S", "group4", "oximes"]
families = ["Amides", "Amidines", "Aromatics", "Aromatics", 
                   "Carbamates", "$C_{x}H_{y}O_{(0,1)}$", "$C_{x}H_{y}O_{(0,1)}$",
                   "$C_{x}H_{y}N$", "$C_{x}H_{y}S$", "$C_{x}H_{y}O_{(2,3)}$", 
                   "Oximes"]

family_dict = dict(zip(all_datasets, families))

In [ ]:
bad_representations = []
bad_items = []
datasets_size = []
dataset_labels = []
for dataset in all_datasets:
    _, wrong_graphs, wrong_samples, tot, labels = geometry_to_graph_analysis(dataset)
    print("----------------------------------------")
    bad_representations.append(wrong_graphs)
    bad_items.append(wrong_samples)
    datasets_size.append(tot)
    dataset_labels.append(labels)

bad_dict = dict(zip(all_datasets, bad_representations))
bad_items_dict = dict(zip(all_datasets, bad_items))
dataset_size_dict = dict(zip(all_datasets, datasets_size))
labels_dict = dict(zip(all_datasets, dataset_labels))
total_bad = sum(bad_representations)
total_datasets = sum(datasets_size)

In [ ]:
labels_dict["amidines"]

In [ ]:
total_size_family = [0] * (len(set(families)))
for i in range(len(total_size_family)):
    for group in all_datasets:
        if family_dict[group] == list(set(families))[i]:
            total_size_family[i] += dataset_size_dict[group]
            
bad_family = [0] * (len(set(families)))
for i in range(len(bad_family)):
    for group in all_datasets:
        if family_dict[group] == list(set(families))[i]:
            bad_family[i] += bad_dict[group]

In [ ]:
bad_family_dict = dict(zip(set(families),bad_family))
total_size_dict = dict(zip(set(families),total_size_family))
families = list(total_size_dict.keys())
bad = list(bad_family_dict.values())
tot = list(total_size_dict.values())

In [ ]:
import pandas as pd

In [ ]:
df_family = pd.DataFrame(data=dict(zip(["Family", "Bad", "Total"], [families, bad, tot])))

In [ ]:
df_family

In [ ]:
print("Graphs with no metal-adsorbate connections: {}/{} ({}%)".format(total_bad, total_datasets, (total_bad / total_datasets) * 100))

In [ ]:

y = np.array(list(bad_family_dict.values()))
y_tot = np.array(list(dataset_size_dict.values()))
order = ["$C_{x}H_{y}O_{(0,1)}$", "$C_{x}H_{y}O_{(2,3)}$", "$C_{x}H_{y}N$", 
         "$C_{x}H_{y}S$", "Amidines", "Amides", 
         "Oximes", "Carbamates", "Aromatics"]
f, ax = plt.subplots(figsize=(8,5), dpi=800)
params = {'mathtext.default': 'regular' }          
plt.rcParams.update(params)
#sns.set_theme(style="whitegrid")
sns.set_color_codes('muted')
sns.barplot(data=df_family,
            x="Total", 
            y="Family", 
            orient="h", 
            label="Total", 
            color="b", 
            order=order)
sns.set_color_codes('pastel')
sns.barplot(data=df_family,
            x="Bad", 
            y="Family",
            orient="h", 
            label="Bad", 
            color="b", 
            order=order)
ax.legend(ncol=1, loc=(0.75, 0.2), frameon=True)
ax.set(xlabel="Samples", ylabel="Family")
sns.despine(left=True, bottom=False)
#plt.savefig("Bad_family.svg")

# Quantitative analysis and distribution of bad representations over metals

Due to different nomenclature convention applied over the datasets not done by myself (Santiago), these sets are treated separately.

## FG-dataset

In [ ]:
santi_datasets = ['amides', 'amidines', 'aromatics', 'aromatics2', 
                  'carbamate_esters', 'group2', 'group2b', 'group3N',
                  'group3S', 'group4', 'oximes']  # Share same nomenclature convention
santi_metals = ["ag", "au", "cd", "cu", 
                "ir", "ni", "os", "pd",
                "pt", "rh", "ru", "zn"]
bad_counter = np.zeros((len(santi_metals)))
total_counter = np.zeros((len(santi_metals)))
for i in santi_datasets:
    for j in bad_items_dict[i]:
        for k in range(len(santi_metals)):
            if j[:2] == santi_metals[k]:
                bad_counter[k] += 1
            else:
                pass
            
for i in santi_datasets:
    for j in labels_dict[i]:
        for k in range(len(santi_metals)):
            if j[:2] == santi_metals[k]:
                total_counter[k] += 1
            else:
                pass

santi_bad_counter = list(bad_counter)
santi_tot_counter = list(total_counter)
santi_bad_counter = [int(i) for i in santi_bad_counter]
santi_bad_dict = dict(zip(santi_metals, santi_bad_counter))
santi_tot_dict = dict(zip(santi_metals, santi_tot_counter))
print(santi_tot_dict)

In [ ]:
x = santi_metals
y = np.array(santi_bad_counter)
z = np.array(santi_tot_counter)

f, ax = plt.subplots(figsize=(8,5), dpi=800)
sns.set_theme(style="whitegrid")
sns.set_color_codes('muted')
sns.barplot(x=z, 
            y=[i.capitalize() for i in x], 
            orient="h", 
            label="Total", 
            color="b")
sns.set_color_codes('pastel')
sns.barplot(x=y, 
            y=[i.capitalize() for i in x], 
            orient="h", 
            label="Bad", 
            color="b")
ax.set(xlabel="Samples", ylabel="Metal")
ax.legend(ncol=1, loc=(0.75, 0.2), frameon=True)

sns.despine(left=True, bottom=False)
#plt.savefig("Bad_metal.svg")